# Function Definitions

In [2]:
def convert_df_to_datetime(df, columns_to_convert):
    for column in columns_to_convert:
        df[column] = pd.to_datetime(df[column])

In [3]:
#turns out to be unnecessary as pd.to_datetime can handle the trailing "UT" in the timestamp string
def clean_timestamp_columns_and_strip_UT(df, columns_to_convert):
    for column in columns_to_convert:
        df[column] = df[column].str.strip("UT")

# Flight Analysis

In [7]:
import pandas as pd
from geopy.distance import distance
from datetime import timedelta
import matplotlib.pyplot as plt
from ipyleaflet import Map, Polyline, basemaps, basemap_to_tiles, LocalTileLayer, Marker, AwesomeIcon, Popup, FullScreenControl
from ipywidgets import HTML, Layout
import random


In [37]:
#pd.read_csv('data/REGA_1.csv', sep=';')

In [8]:
heli = pd.read_csv('data/REGA_1.csv', sep=';').dropna()
heli['Zeit Status DP'] = pd.to_datetime(heli['Zeit Status DP'])
heli['Zeit Status 1'] = pd.to_datetime(heli['Zeit Status 1'])
heli['Zeit Status 2'] = pd.to_datetime(heli['Zeit Status 2'])
heli['Zeit Status 3'] = pd.to_datetime(heli['Zeit Status 3'])
heli['Zeit Status 4'] = pd.to_datetime(heli['Zeit Status 4'])
heli

c:\Users\KshitizGupta\AppData\Local\Programs\Python\Python39\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname UT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
c:\Users\KshitizGupta\AppData\Local\Programs\Python\Python39\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname UT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
c:\Users\KshitizGupta\AppData\Local\Programs\Python\Python39\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname UT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-awar

,Disponummer,Einsatznummer,Einheit,Zeit Status DP,Latitude Disposition,Longitude Disposition,Zeit Status 1,Latitude Status 1,Longitude Status 1,Zeit Status 2,Latitude Status 2,Longitude Status 2,Zeit Status 3,Latitude Status 3,Longitude Status 3,Zeit Status 4,Latitude Status 4,Longitude Status 4
0,D22087458,S22122175,REGA_1,2022-08-15 15:08:09,47.395956,8.637702,2022-08-15 15:09:09,47.395956,8.637702,2022-08-15 15:21:20,47.395956,8.637702,2022-08-15 15:52:30,47.395956,8.637702,2022-08-15 16:15:05,47.372134,8.557430
1,D22087519,S22122274,REGA_1,2022-08-15 18:51:22,47.395956,8.637702,2022-08-15 18:53:04,47.395956,8.637702,2022-08-15 19:04:02,47.395956,8.637702,2022-08-15 19:26:51,47.395956,8.637702,2022-08-15 19:45:26,47.429844,9.393868
2,D22087785,S22122672,REGA_1,2022-08-16 13:50:16,47.395956,8.637702,2022-08-16 13:51:30,47.395956,8.637702,2022-08-16 14:07:10,47.119875,8.606510,2022-08-16 14:23:19,47.119875,8.606510,2022-08-16 14:31:29,47.066392,8.327905
3,D22087833,S22122752,REGA_1,2022-08-16 16:17:02,47.395956,8.637702,2022-08-16 16:18:38,47.395956,8.637702,2022-08-16 16:36:38,47.684625,8.621056,2022-08-16 16:47:16,47.684625,8.621056,2022-08-16 17:02:23,47.507378,8.727538
4,D22087855,S22122794,REGA_1,2022-08-16 17:25:42,47.395956,8.637702,2022-08-16 17:27:26,47.395956,8.637702,2022-08-16 17:39:47,47.395956,8.637702,2022-08-16 18:19:54,47.395956,8.637702,2022-08-16 18:36:06,47.429902,9.394118
5,D22088177,S22123290,REGA_1,2022-08-17 15:04:30,47.395956,8.637702,2022-08-17 15:05:35,47.395956,8.637702,2022-08-17 15:21:33,47.701623,8.626231,2022-08-17 15:44:06,47.701623,8.626231,2022-08-17 15:55:31,47.372134,8.557430
6,D22088461,S22123757,REGA_1,2022-08-18 11:45:16,47.395956,8.637702,2022-08-18 11:46:42,47.395956,8.637702,2022-08-18 12:02:55,47.713075,8.639820,2022-08-18 12:31:49,47.711762,8.635448,2022-08-18 12:43:40,47.366115,8.496674
7,D22088582,S22123937,REGA_1,2022-08-18 17:59:57,47.395956,8.637702,2022-08-18 18:01:21,47.395956,8.637702,2022-08-18 18:13:48,47.593445,8.490966,2022-08-18 18:25:19,47.593445,8.490966,2022-08-18 18:33:24,47.377310,8.550528
10,D22089197,S22124927,REGA_1,2022-08-20 13:30:42,47.395956,8.637702,2022-08-20 13:31:32,47.395956,8.637702,2022-08-20 13:41:27,47.447037,8.417233,2022-08-20 14:04:06,47.447720,8.409005,2022-08-20 14:08:57,47.377310,8.550528
11,D22089470,S22125323,REGA_1,2022-08-21 09:50:17,47.395956,8.637702,2022-08-21 09:51:44,47.395956,8.637702,2022-08-21 09:58:29,47.452573,8.629110,2022-08-21 10:13:09,47.456475,8.627528,2022-08-21 10:22:36,47.506870,8.728281


In [9]:
einsatz = "S22122175"
    # , "S22122274"
# einsatz = input("Einsatz: ")

# flug = heli[heli['Einsatznummer'] == einsatz]  center = (47.39595583, 8.637702055)
flug = heli
if flug.empty:
    raise ValueError("Eingegebene Einsatznummer nicht gefunden.")
m = Map(center = (47.39595583, 8.637702055), zoom=11, layout=Layout(width='100%', height='1050px'), scroll_wheel_zoom=True)
for i in range(heli.shape[0]):
    print(i)
    zeitpunkte = flug[['Zeit Status DP', 'Zeit Status 1', 'Zeit Status 2', 'Zeit Status 3', 'Zeit Status 4']].iloc[i]
    lats = \
        flug[['Latitude Disposition', 'Latitude Status 1', 'Latitude Status 2', 'Latitude Status 3',
              'Latitude Status 4']].iloc[
            i].values
    lons = flug[['Longitude Disposition', 'Longitude Status 1', 'Longitude Status 2', 'Longitude Status 3',
                 'Longitude Status 4']].iloc[i].values

    latlons = list(zip(lats, lons))
    zeitpunkte = [zp.strftime("%H:%M:%S") for zp in zeitpunkte]
    message0 = HTML()
    message1 = HTML()
    message2 = HTML()
    message3 = HTML()
    message4 = HTML()
    message0.value = ("<b>Disponiert: </b>") + zeitpunkte[0]
    message1.value = ("<b>Ausgerückt: </b><br>") + zeitpunkte[1]
    message2.value = ("<b>Am Einsatzort: </b><br>") + zeitpunkte[2]
    message3.value = ("<b>Transport Patient: </b><br>") + zeitpunkte[3]
    message4.value = ("<b>Am Zielort: </b><br>") + zeitpunkte[4]

    #adding functionality for different color in the lines of Polyline
    r = lambda: random.randint(0, 255)
    colour = '#%02X%02X%02X' % (r(), r(), r())

    line = Polyline(
        locations=latlons,
        color=colour,
        # color="magenta",
        fill=False
    )

    popup_0 = Popup(
        location=latlons[0],
        child=message1,
        close_button=False,
        auto_close=False,
        close_on_escape_key=False
    )

    popup_1 = Popup(
        location=latlons[1],
        child=message1,
        close_button=False,
        auto_close=False,
        close_on_escape_key=True
    )

    popup_2 = Popup(
        location=latlons[2],
        child=message2,
        close_button=False,
        auto_close=False,
        close_on_escape_key=True
    )

    popup_3 = Popup(
        location=latlons[3],
        child=message3,
        close_button=False,
        auto_close=False,
        close_on_escape_key=True
    )

    popup_4 = Popup(
        location=latlons[4],
        child=message4,
        close_button=False,
        auto_close=False,
        close_on_escape_key=True
    )

    start = AwesomeIcon(
        name='home',
        marker_color='blue',
        icon_color='darkblue',
    )

    ziel = AwesomeIcon(
        name='h-square',
        marker_color='blue',
        icon_color='darkblue',
    )

    eo = AwesomeIcon(
        name='heartbeat',
        marker_color='blue',
        icon_color='darkblue',
    )

    route = AwesomeIcon(
        name='road',
        marker_color='blue',
        icon_color='darkblue',
    )

    marker1 = Marker(icon=start, title="Ausgerückt", location=latlons[1], draggable=False, popup=message1)
    marker2 = Marker(icon=ziel, title="Am Einsatzort", location=latlons[4], draggable=False, popup=message4)
    marker3 = Marker(icon=eo, title="Transport Patient", location=latlons[2], draggable=False, popup=message2)
    marker4 = Marker(icon=route, title="Am Zielort", location=latlons[3], draggable=False, popup=message3)

    # Initialisierung Karte

    m.add_layer(LocalTileLayer(path='https://tile.osm.ch/switzerland/{z}/{x}/{y}.png'))
    m.add_layer(marker1)
    m.add_layer(marker2)
    m.add_layer(marker3)
    m.add_layer(marker4)
    m.add_layer(line)
    # m.add_layer(popup_0)
    # m.add_layer(popup_1)
    # m.add_layer(popup_2)
    # m.add_layer(popup_3)
    # m.add_layer(popup_4)


# %%
# Export nach HTML
m.add_control(FullScreenControl())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


In [40]:
# Export nach HTML
m.save('analysis.html', title='Fluganalyse')

# Kartenanbieter

OSM Standard Swiss
https://tile.osm.ch/switzerland/{z}/{x}/{y}.png

SRZ Opentopomap Dark
m.add_layer(LocalTileLayer(path='http://elsm1451.els.loc/srz-dark/{z}/{x}/{y}.png'))

SRZ Opentopomap Light
m.add_layer(LocalTileLayer(path='http://elsm1451.els.loc/opentopomap/{z}/{x}/{y}.png'))

OpenTopoMap
opentopomap = basemap_to_tiles(basemaps.OpenTopoMap)
m.add_layer(opentopomap)

Swisstopo Basemap
swisstopo = basemap_to_tiles(basemaps.SwissFederalGeoportal.NationalMapColor)
m.add_layer(swisstopo)

Swisstopo Light Base Map
m.add_layer(LocalTileLayer(path='https://api.maptiler.com/maps/ch-swisstopo-lbm/{z}/{x}/{y}.png?key=62fWxjuKZdoP6vlFjq0a'))

Swisstopo Light Base Map Dark
m.add_layer(LocalTileLayer(path='https://api.maptiler.com/maps/ch-swisstopo-lbm-dark/{z}/{x}/{y}@2x.png?key=62fWxjuKZdoP6vlFjq0a'))

Swisstopo Light Base Map Vivid
m.add_layer(LocalTileLayer(path='https://api.maptiler.com/maps/ch-swisstopo-lbm-vivid/256/{z}/{x}/{y}.png?key=62fWxjuKZdoP6vlFjq0a'))

MapTiler Bright
m.add_layer(LocalTileLayer(path='https://api.maptiler.com/maps/bright/256/{z}/{x}/{y}.png?key=62fWxjuKZdoP6vlFjq0a'))

Swiss Style SOSM
m.add_layer(LocalTileLayer(path='https://tile.osm.ch/osm-swiss-style/{z}/{x}/{y}.png'))

Streets Style
m.add_layer(LocalTileLayer(path='https://api.maptiler.com/maps/streets/{z}/{x}/{y}@2x.png?key=62fWxjuKZdoP6vlFjq0a'))



In [41]:
#m = Map(center=(47.25983648568554, 8.71772586165636), zoom=9)
#m.add_layer(LocalTileLayer(path='https://api.maptiler.com/maps/ch-swisstopo-lbm/{z}/{x}/{y}.png?key=62fWxjuKZdoP6vlFjq0a'))
#m
